## S5_LGB_main.py

In [1]:
import warnings
warnings.simplefilter("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc,os,random
import time,datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from utils import *
from model import *
root = args.root
seed = args.seed

available gpus: [0]


In [2]:
df = pd.read_feather(f"{root}/all_feature.feather")

In [3]:
train_y = pd.read_csv(f"/mnt/sdb/KAGGLE_DATA/amex-default-prediction/train_labels.csv")

In [4]:
train = df[:train_y.shape[0]]
train["target"] = train_y["target"]
test = df[train_y.shape[0]:].reset_index(drop=True)
del df; gc.collect()

0

In [5]:
print(train.shape, test.shape)

(458913, 6390) (924621, 6389)


In [6]:
lgb_config = {
    "lgb_params": {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting": "dart",
        "max_depth": -1,
        "num_leaves": 64,
        "learning_rate": 0.035,
        "bagging_freq": 5,
        "bagging_fraction": 0.75,
        "feature_fraction": 0.05,
        "min_data_in_leaf": 256,
        "max_bin": 63,
        "min_data_in_bin": 256,
        #"min_sum_hessian_in_leaf": 10,
        "tree_learner": "serial",
        "boost_from_average": "false",
        "lambda_l1": 0.1,
        "lambda_l2": 30,
        "num_threads": 16,
        "verbosity": 1
    },
    "feature_name": [],
    "rounds": 4500,
    "early_stopping_rounds": 100,
    "verbose_eval": 50,
    "folds": 5,
    "seed": seed,
}

### trainとtest同時にやろうとするとメモリ終わるので、分けてやる必要アリ
* さらにtestは1発だとメモリ乗り切らないのでわけないといけない

In [7]:
batch_num = 5
batch_size = test.shape[0] // batch_num
batch_amari = test.shape[0] - batch_size*batch_num

In [8]:
BATCH_ID = 4
assert BATCH_ID < batch_num, "BATCH_ID must be lower than batch_num"

In [9]:
if BATCH_ID == (batch_num-1):
    test = test.iloc[batch_size*BATCH_ID:(batch_size*(BATCH_ID+1) + batch_amari)]
else:
    test = test.iloc[batch_size*BATCH_ID:(batch_size*(BATCH_ID+1))]

In [10]:
lgb_config['feature_name'] = [col for col in train.columns if col not in [id_name,label_name,'S_2'] and 'target' not in col]
del train; gc.collect()

0

In [11]:
Lgb_train_and_predict(train=None, test=test, config=lgb_config, test_batch_id=BATCH_ID, aug=None, run_id='LGB_with_manual_feature')

cp: cannot stat './*.sh': No such file or directory


 Lgb_train_and_predict : LGB_with_manual_feature


100%|██████████| 5/5 [01:48<00:00, 21.68s/it]


(None,
                                               customer_ID  prediction
 739696  cccebdb0ebf7f3c98c5b8d72a9375f7ff6fe09f123b668...    0.060926
 739697  cccebec87d75006aa0ce200f2a8173d861ecdf5a8b6a1a...    0.629057
 739698  ccced959647957553808df60a4de7719a8c04853570b6d...    0.573951
 739699  cccf12af1b620b95393d292b337abf65b30cebaaa89c55...    0.590253
 739700  cccf313ba6a4885cc8b68cbf4af5d38ba1942d173f0195...    0.948561
 ...                                                   ...         ...
 924616  ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...    0.011252
 924617  ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...    0.865559
 924618  ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...    0.380820
 924619  ffffddef1fc3643ea179c93245b68dca0f36941cd83977...    0.235823
 924620  fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...    0.057372
 
 [184925 rows x 2 columns],
 (None, None))

In [12]:
## lgb_config['feature_name'] = [col for col in train.columns if col not in [id_name,label_name,'S_2']]
## Lgb_train_and_predict(train,test,lgb_config,aug=None,run_id='LGB_with_manual_feature_and_series_oof')